In [ ]:
%load_ext autoreload
%autoreload 2

import pickle
import polars as pl
import matplotlib.pyplot as plt
from pre_snap_prediction.modeling import orpsp_classification

In [2]:
orpsp_features = pl.read_csv("../data/orpsp_features.csv", null_values=["NA", ""])
orpsp_target = pl.read_csv("../data/orpsp_target.csv", null_values=["NA", ""])

In [3]:
encoder = pickle.load(open("../models/encoder.pkl", "rb"))
classification_model = pickle.load(open("../models/classification_model.pkl", "rb"))

In [4]:
orpsp_features = orpsp_classification.select_orpsp_features(orpsp_features)

In [5]:
orpsp_features_encoded = orpsp_classification.transform_encoder(orpsp_features, encoder)

In [6]:
x_train, x_test, y_train, y_test = orpsp_classification.orpsp_train_test_split(orpsp_features_encoded, orpsp_target)

In [7]:
y_pred = orpsp_classification.predict_orpsp_class(x_test, classification_model)
y_train_pred = orpsp_classification.predict_orpsp_class(x_train, classification_model)

In [8]:
y_pred_proba = orpsp_classification.predict_orpsp(x_test, classification_model)
y_train_pred_proba = orpsp_classification.predict_orpsp(x_train, classification_model)

In [ ]:
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report, roc_curve, auc

print("Train: " + str(balanced_accuracy_score(y_train, y_train_pred)))
print("Test: " + str(balanced_accuracy_score(y_test, y_pred)))

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))
ConfusionMatrixDisplay(confusion_matrix(y_train, y_train_pred), display_labels=["No Reception", "Received Pass"]).plot(ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))
ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred), display_labels=["No Reception", "Received Pass"]).plot(ax=ax)

In [ ]:
print(classification_report(y_train, y_train_pred))

In [ ]:
print(classification_report(y_test, y_pred))

In [14]:
fpr_test, tpr_test, thresholds_test = roc_curve(y_test, y_pred_proba["orpsp"].to_numpy())
fpr_train, tpr_train, thresholds_train = roc_curve(y_train, y_train_pred_proba["orpsp"].to_numpy())

In [ ]:
plt.plot(fpr_train, tpr_train, label=f"Train AUC = {auc(fpr_train, tpr_train):.2f}")
plt.plot(fpr_test, tpr_test, label=f"Test AUC = {auc(fpr_test, tpr_test):.2f}")
plt.legend()

In [ ]:
from shapash import SmartExplainer

In [17]:
xpl = SmartExplainer(
    model=classification_model,
)

In [ ]:
xpl.compile(
    x_test.to_pandas().drop(columns=["gameId", "playId", "nflId"]),
    y_target=y_test.to_pandas(),
)

In [ ]:
xpl.plot.features_importance(mode='global-local', max_features=30, zoom=True)

In [ ]:
app = xpl.run_app(port="8051")

In [ ]:
app.kill()